In [1]:
from owslib.wms import WebMapService
import tqdm
import numpy as np
import io
from PIL import Image
import sys

INDEX = 0

wms_map_list = [
    'https://kartta.hel.fi/ws/geoserver/avoindata/wms?request=getCapabilities',
    'https://kartta.luke.fi/geoserver/MVMI/wms?version=1.3.0',
    'https://paikkatieto.ymparisto.fi/arcgis/services/INSPIRE/SYKE_Maanpeite/MapServer/WMSServer?request=GetCapabilities&service=WMS',
]

# NOTE: set different sources here
hel_map = WebMapService(wms_map_list[INDEX])

# NOTE: set different layers here
def layers_filter(hel_map, index):
    if index == 0:  # hel.fi
        return [
            name for name in hel_map.contents
            if ('rtoilmakuva' in name and ('19' in name or '20' in name))
        ]
    elif index == 1:  # luke.fi
        return [name for name in hel_map.contents if ('kuusi' in name)]
    elif index == 2:  # ymparisto.fi
        pass

# get different bounding boxes
def get_submap(points, step=0.001, precision=14):
    point1 = points[0]
    point2 = points[1]
    for i in range(int(abs((point2[0] - point1[0])) / step)):
        for j in range(int(abs((point2[1] - point1[1])) / step)):
            a = np.around(point1[0] + i * step, precision)
            b = np.around(point1[1] + j * step, precision)
            c = np.around(point1[0] + (i + 1) * step, precision)
            d = np.around(point1[1] + (j + 1) * step, precision)
            yield (a, b, c, d)

# is this image blank (255)
def is_image_useful(image_flow):
    memory_file = io.BytesIO()
    memory_file.write(image_flow.read())
    image = Image.open(memory_file)
    gray_image = np.array(image.convert('L'))
    if gray_image.mean() < 245:
        return True
    else:
        return False


print(
    f'Title: {hel_map.identification.title}\nVersion: {hel_map.identification.version}\nAbstract: {hel_map.identification.abstract}'
)


Title: Helsinki_WMS
Version: 1.1.1
Abstract: Helsingin kaupunkimittauspalveluiden ylläpitämä WMS-rajapintapalvelu internet-käyttäjille.


In [2]:
layer_list = layers_filter(hel_map, INDEX)

# get info we need
info_list = []
for layer in layer_list:
    print(
        f' - Title: {hel_map[layer].title}\n - Range: {hel_map[layer].boundingBoxWGS84}\n - crsOptions: {hel_map[layer].crsOptions}\n - Styles: {hel_map[layer].styles}\n'
    )
    info_list.append({
        'layer': layer,
        'box': hel_map[layer].boundingBoxWGS84,
        'crs': hel_map[layer].crsOptions,
        'style': list(hel_map[layer].styles.keys())
    })

 - Title: Ortoilmakuva_2022_5cm
 - Range: (24.764732945863038, 60.008010779565915, 25.289558715996222, 60.322459882254456)
 - crsOptions: ['EPSG:3884', 'EPSG:3874', 'EPSG:2393', 'EPSG:3875', 'EPSG:3885', 'EPSG:3387', 'EPSG:2391', 'EPSG:3878', 'EPSG:3879', 'EPSG:4258', 'EPSG:3067', 'EPSG:2392', 'EPSG:3881', 'EPSG:3047', 'EPSG:3883', 'EPSG:3048', 'EPSG:3386', 'EPSG:2394', 'EPSG:3873', 'EPSG:3877', 'EPSG:3882', 'EPSG:3046', 'EPSG:4326', 'EPSG:3857', 'EPSG:3880', 'EPSG:4123', 'EPSG:3876']
 - Styles: {'default-style-avoindata:06_ortoilmakuvat': {'title': 'avoindata:06_ortoilmakuvat style', 'legend': 'https://kartta.hel.fi/ws/geoserver/avoindata/wms?request=GetLegendGraphic&version=1.1.1&format=image%2Fpng&width=20&height=20&layer=avoindata%3A06_ortoilmakuvat'}, 'rasteri': {'title': 'rasteri', 'legend': 'https://kartta.hel.fi/ws/geoserver/avoindata/wms?request=GetLegendGraphic&version=1.1.1&format=image%2Fpng&width=20&height=20&layer=avoindata%3AOrtoilmakuva_2022_5cm'}}

 - Title: Ortoilmaku

In [3]:
# [op.name for op in hel_map.operations]
# hel_map.getOperationByName('GetMap').methods
# hel_map.getOperationByName('GetMap').formatOptions

In [4]:
# two bounding boxes, larger and smaller
hel_range = ((24.461826, 59.965919), (25.518606, 60.444763))
reduced_hel_range = ((24.9, 60), (25.2, 60.3))
# 
for info in tqdm.tqdm(info_list):
    for srs_index in range(27):
        for box in get_submap(reduced_hel_range, 0.01):
            # print(
            #     f'Laying {info["layer"]} on {box}, style: {info["style"][0]}, srs: {info["crs"][0]}'
            # )
            img = hel_map.getmap(
                layers=[info['layer']],
                srs=info['crs'][srs_index],
                bbox=(24.9, 60, 25.2, 60.3),
                # size=(5120, 5120),
                size=(256, 256),
                format='image/png',
                transparent=False)
            if is_image_useful(img):
                print(f'Image {info["layer"]}_{box}_{srs_index} is useful, saving...')
                with open(f'./data/{info["layer"]}_{box}_{srs_index}.png', 'wb') as f:
                    f.write(img.read())
            else:
                print(f'Image {info["layer"]}_{box}_{srs_index} is useless')
        break
    break

  0%|          | 0/61 [00:00<?, ?it/s]

Image avoindata:Ortoilmakuva_2022_5cm_(24.9, 60.0, 24.91, 60.01)_0 is useless
Image avoindata:Ortoilmakuva_2022_5cm_(24.9, 60.01, 24.91, 60.02)_0 is useless
Image avoindata:Ortoilmakuva_2022_5cm_(24.9, 60.02, 24.91, 60.03)_0 is useless
Image avoindata:Ortoilmakuva_2022_5cm_(24.9, 60.03, 24.91, 60.04)_0 is useless
Image avoindata:Ortoilmakuva_2022_5cm_(24.9, 60.04, 24.91, 60.05)_0 is useless
Image avoindata:Ortoilmakuva_2022_5cm_(24.9, 60.05, 24.91, 60.06)_0 is useless
Image avoindata:Ortoilmakuva_2022_5cm_(24.9, 60.06, 24.91, 60.07)_0 is useless
Image avoindata:Ortoilmakuva_2022_5cm_(24.9, 60.07, 24.91, 60.08)_0 is useless
Image avoindata:Ortoilmakuva_2022_5cm_(24.9, 60.08, 24.91, 60.09)_0 is useless
Image avoindata:Ortoilmakuva_2022_5cm_(24.9, 60.09, 24.91, 60.1)_0 is useless
Image avoindata:Ortoilmakuva_2022_5cm_(24.9, 60.1, 24.91, 60.11)_0 is useless
Image avoindata:Ortoilmakuva_2022_5cm_(24.9, 60.11, 24.91, 60.12)_0 is useless
Image avoindata:Ortoilmakuva_2022_5cm_(24.9, 60.12, 24.

  0%|          | 0/61 [00:59<?, ?it/s]

Image avoindata:Ortoilmakuva_2022_5cm_(25.19, 60.25, 25.2, 60.26)_0 is useless
Image avoindata:Ortoilmakuva_2022_5cm_(25.19, 60.26, 25.2, 60.27)_0 is useless
Image avoindata:Ortoilmakuva_2022_5cm_(25.19, 60.27, 25.2, 60.28)_0 is useless
Image avoindata:Ortoilmakuva_2022_5cm_(25.19, 60.28, 25.2, 60.29)_0 is useless


In [ ]:
# NOTE: it seems that the server will only return the whole image in Helsinki area. I cannot get a hier resolution image.

hel_range = ((24.461826, 59.965919), (25.518606, 60.444763))
reduced_hel_range = ((24.9, 60), (25.2, 60.3))
useful_srs = []
useful_srs_index = [5, 12, 17]
for info in tqdm.tqdm(info_list):
    for srs_index in useful_srs_index:
        for box in get_submap(reduced_hel_range, 0.001):
            # print(
            #     f'Laying {info["layer"]} on {box}, style: {info["style"][0]}, srs: {info["crs"][0]}'
            # )
            img = hel_map.getmap(
                layers=[info['layer']],
                srs=info['crs'][srs_index],
                bbox=(24.9, 60, 25.2, 60.3),
                # size=(5120, 5120),
                size=(1024, 1024),
                format='image/png',
                transparent=False)
            if is_image_useful(img):
                print(
                    f'Image {info["layer"]}_{box}_{srs_index} is useful, saving...'
                )
                useful_srs.append(info['crs'][srs_index])
                with open(f'./data/{info["layer"]}_{box}_{srs_index}.png',
                          'wb') as f:
                    f.write(img.read())
            else:
                print(f'Image {info["layer"]}_{box}_{srs_index} is useless',flush=True)


In [ ]:
list(set(useful_srs))